In [26]:
!chainlit hello

2024-06-17 13:49:17 - Your app is available at http://localhost:8000


ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted


In [24]:
import os
from chainlit import on_chat_start , on_message
from langchain import HuggingFaceHub, PromptTemplate, LLMChain

In [13]:
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

In [14]:
#model_id = "microsoft/DialoGPT-medium" : conversational models are not currently supported by Langchain
#model_id = "mosaicml/mpt-7b-instruct"
#model_id = "tiiuae/falcon-7b"
model_id = "gpt2-medium"  #355M parameters
conv_model = HuggingFaceHub(huggingfacehub_api_token=os.environ['HUGGINGFACEHUB_API_TOKEN'],
                            repo_id=model_id,
                            model_kwargs={"temperature":0.8, "max_new_tokens":200}) #0 to 1

In [15]:
template = """You are a helpful AI assistant that makes stories by completing the query provided by the user 

{query}
"""

prompt = PromptTemplate(template=template, input_variables=['query'])

In [16]:
conv_chain = LLMChain(llm=conv_model,
                      prompt=prompt,
                      verbose=True)

In [27]:
print(conv_chain.run("Give me a good receipe to make a cup cake."))



> Entering new LLMChain chain...
Prompt after formatting:
You are a helpful AI assistant that makes stories by completing the query provided by the user 

Give me a good receipe to make a cup cake.


> Finished chain.
You are a helpful AI assistant that makes stories by completing the query provided by the user 

Give me a good receipe to make a cup cake.

I get tired when answering questions, but I get better when chatting with you.

I would like to make a request at your place!

I don't need to answer a lot of questions but I am willing to help you with certain things.

All I need is a friendly and helpful smile and I'll be happy to act as your assistant.

I would like to buy something from your shop.

I want to know how much I should buy for my delivery.

I would like to maintain a company for a while.


In [25]:
@on_chat_start
def main():
    prompt = PromptTemplate(template=template, input_variables=['query'])
    conv_chain = LLMChain(llm=conv_model,
                          prompt=prompt,
                          verbose=True)
    
    cl.user_session.set("llm_chain", conv_chain)
    
@on_message
async def main(message:str):
    llm_chain = cl.user_session.get("llm_chain")
    res = await llm_chain.acall(message, callbacks=[cl.AsyncLangchainCallbackHandler()])
    
    #perform post processing on the received response here
    #res is a dict and the response text is stored under the key "text"
    await cl.Message(content=res["text"]).send()